In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import os

def generate_synthetic_data(n_samples=1000, n_features=4, n_classes=3, test_size=0.2, random_state=42):
    """Generate synthetic data for classification with some non-linear relationships."""
    np.random.seed(random_state)
    
    # Generate features
    X = np.random.randn(n_samples, n_features)
    
    # Create non-linear relationships
    X[:, 0] = np.sin(X[:, 0]) + np.random.randn(n_samples) * 0.1
    X[:, 1] = np.exp(X[:, 1] / 2) + np.random.randn(n_samples) * 0.1
    X[:, 2] = X[:, 0] * X[:, 1] + np.random.randn(n_samples) * 0.1
    X[:, 3] = np.square(X[:, 3]) + np.random.randn(n_samples) * 0.1
    
    # Generate target classes based on non-linear combinations
    logits = np.zeros((n_samples, n_classes))
    logits[:, 0] = np.sin(X[:, 0]) + np.cos(X[:, 1])
    logits[:, 1] = X[:, 2] * X[:, 3] - np.square(X[:, 1])
    logits[:, 2] = np.exp(X[:, 0]/2) - np.sin(X[:, 2] * X[:, 3])
    
    # Convert to probabilities and select class
    probs = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
    y = np.argmax(probs, axis=1)
    
    # Standardize features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # Create DataFrames
    feature_cols = [f'feature_{i}' for i in range(n_features)]
    df = pd.DataFrame(X, columns=feature_cols)
    df['target'] = y
    
    # Split into train and validation sets
    mask = np.random.rand(len(df)) < (1 - test_size)
    train_df = df[mask]
    val_df = df[~mask]
    
    return train_df, val_df

def main():
    # Create input_data directory if it doesn't exist
    os.makedirs('input_data', exist_ok=True)
    
    # Generate data
    train_df, val_df = generate_synthetic_data(
        n_samples=1000,
        n_features=4,
        n_classes=3,
        test_size=0.2,
        random_state=42
    )
    
    # Save to CSV files
    train_df.to_csv('input_data/train.csv', index=False)
    val_df.to_csv('input_data/val.csv', index=False)
    
    print(f"Generated {len(train_df)} training samples and {len(val_df)} validation samples")
    print("\nFeature statistics:")
    print(train_df.describe())
    print("\nClass distribution in training set:")
    print(train_df['target'].value_counts(normalize=True))

if __name__ == "__main__":
    main()


Generated 793 training samples and 207 validation samples

Feature statistics:
        feature_0   feature_1   feature_2   feature_3      target
count  793.000000  793.000000  793.000000  793.000000  793.000000
mean     0.015966   -0.006794    0.017763   -0.002376    1.543506
std      0.991243    1.017013    0.998462    1.026261    0.829346
min     -1.823993   -1.676095   -4.557125   -0.936869    0.000000
25%     -0.841765   -0.699470   -0.588484   -0.625870    2.000000
50%      0.065698   -0.197729    0.020473   -0.391615    2.000000
75%      0.931161    0.365627    0.598809    0.266843    2.000000
max      1.768643    8.748089    4.253251   10.146557    2.000000

Class distribution in training set:
target
2    0.762926
0    0.219420
1    0.017654
Name: proportion, dtype: float64


In [1]:
import random
import os
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm
import logging

import seaborn as sns
import matplotlib.pyplot as plt
import yaml
from sklearn.metrics import f1_score


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import optuna
import psutil
import cpuinfo
import intel_extension_for_pytorch as ipex
import multiprocessing


In [2]:
def load_config(config_path):
    """Load configuration from a YAML file."""
    with open(config_path, 'r') as f:
        return yaml.safe_load(f)
        
# Set up logging
def setup_logger(name='MLPTrainer'):
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)
    
    # File handler
    fh = logging.FileHandler(f'{name}.log')
    fh.setLevel(logging.INFO)
    
    # Console handler
    ch = logging.StreamHandler(sys.stdout)
    ch.setLevel(logging.INFO)
    
    # Formatter
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    ch.setFormatter(formatter)
    
    # Add handlers to logger
    logger.addHandler(fh)
    logger.addHandler(ch)
    
    return logger

class CustomDataset(Dataset):
    def __init__(self, df, target_column):
        self.features = torch.FloatTensor(df.drop(target_column, axis=1).values)
        self.labels = torch.LongTensor(df[target_column].values)
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_layers, num_classes=3, dropout_rate=0.2, use_batch_norm=True):
        super(MLPClassifier, self).__init__()
        
        layers = []
        prev_size = input_size
        
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(prev_size, hidden_size))
            if use_batch_norm:
                layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            prev_size = hidden_size
        
        layers.append(nn.Linear(prev_size, num_classes))
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)
        
class PyTorchTrainer:
    """A generic PyTorch trainer class.
    
    Attributes:
        model: PyTorch model to train
        criterion: Loss function
        optimizer: Optimization algorithm
        device: Device to train on (CPU/GPU)
        train_loader: DataLoader for training data
        val_loader: DataLoader for validation data
    """
    
    def __init__(self, model, criterion, optimizer, device='cpu', verbose=False):
        self.model = model.to(device)
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = device
        self.verbose = verbose
        
    def train_epoch(self, train_loader):
        """Trains the model for one epoch."""
        self.model.train()
        total_loss = 0
        correct = 0
        total = 0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(self.device), batch_y.to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(batch_X)
            loss = self.criterion(outputs, batch_y)
            loss.backward()
            self.optimizer.step()
            
            total_loss += loss.item()
            # Save the best model and optimizer from hyperparameter tuning. Reload this best model after hyperparameter tuning. apply the reloaded model to the validation dataset as the final step, to compare its performance with the results of the train_final_model step.
            _, predicted = torch.max(outputs.data, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()
        
        accuracy = 100 * correct / total
        return total_loss / len(train_loader), accuracy
    
    def evaluate(self, val_loader):
        """Evaluates the model on validation data."""
        self.model.eval()
        total_loss = 0
        all_preds = []
        all_labels = []
        correct = 0
        total = 0
        
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(self.device), batch_y.to(self.device)
                outputs = self.model(batch_X)
                loss = self.criterion(outputs, batch_y)
                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += batch_y.size(0)
                correct += (predicted == batch_y).sum().item()
                
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(batch_y.cpu().numpy())
        
        accuracy = 100 * correct / total
        f1 = f1_score(all_labels, all_preds, average='weighted')
        return total_loss / len(val_loader), accuracy, f1

    def train(self, train_loader, val_loader, epochs, metric='accuracy'):
        """Trains the model for specified number of epochs. 
        Monitors specified validation metric for early stopping."""
        train_losses, val_losses = [], []
        train_metrics, val_metrics = [], []
        best_val_metric = 0
        
        for epoch in tqdm(range(epochs), desc='Training'):
            train_loss, train_accuracy = self.train_epoch(train_loader)
            val_loss, val_accuracy, val_f1 = self.evaluate(val_loader)
            
            # Select metric based on config
            train_metric = train_accuracy
            val_metric = val_f1 if metric == 'f1' else val_accuracy
            
            train_losses.append(train_loss)
            val_losses.append(val_loss)
            train_metrics.append(train_metric)
            val_metrics.append(val_metric)
            
            best_val_metric = max(best_val_metric, val_metric)
            
            if self.verbose:
                metric_name = 'F1' if metric == 'f1' else 'Accuracy'
                metric_value = val_f1 if metric == 'f1' else val_accuracy
                print(f'Epoch {epoch+1}/{epochs}: Val {metric_name}: {metric_value:.2f}%')
        
        self.plot_learning_curves(train_losses, val_losses, train_metrics, val_metrics, 
                                metric_name='F1-Score' if metric == 'f1' else 'Accuracy')
        
        return train_losses, val_losses, train_metrics, val_metrics, best_val_metric
    
    @staticmethod
    def plot_learning_curves(train_losses, val_losses, train_metrics, val_metrics, metric_name='Accuracy'):
        """Plots the learning curves for loss and chosen metric (accuracy or F1)."""
        plt.figure(figsize=(10, 6))
        sns.set_style("whitegrid")
        
        # Normalize values for better visualization
        max_loss = max(max(train_losses), max(val_losses))
        max_metric = max(max(train_metrics), max(val_metrics))
        
        epochs = range(1, len(train_losses) + 1)
        
        sns.lineplot(data={
            f"Training {metric_name}": [x/max_metric for x in train_metrics],
            f"Validation {metric_name}": [x/max_metric for x in val_metrics],
            "Training Loss": [x/max_loss for x in train_losses],
            "Validation Loss": [x/max_loss for x in val_losses]
        })
        
        plt.xlabel("Epoch")
        plt.ylabel("Normalized Value")
        plt.title(f"Training and Validation Loss and {metric_name} Curves")
        plt.legend()
        plt.savefig('learning_curves.png')
        plt.close()

class HyperparameterTuner:
    def __init__(self, config):
        self.config = config
        self.best_trial_value = float('-inf')
        self.best_model_state = None
        self.best_optimizer_state = None
        self.best_params = None
        os.makedirs(os.path.dirname(config['model']['save_path']), exist_ok=True)
    
    def save_best_model(self, model, optimizer, trial_value, params):
        """Save the best model and its metadata."""
        checkpoint = {
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'optimizer_name': self.config['training']['optimizer_choice'],
            'metric_name': self.config['training']['optimization_metric'],
            'metric_value': trial_value,
            'hyperparameters': params
        }
        torch.save(checkpoint, self.config['model']['save_path'])
    
    def create_model_and_optimizer(self, trial):
        # Extract hyperparameters from trial
        hidden_layers = []
        n_layers = trial.suggest_int('n_layers', 1, 4)
        for i in range(n_layers):
            hidden_layers.append(trial.suggest_int(f'hidden_layer_{i}', 32, 512))
        
        lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
        dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
        use_batch_norm = trial.suggest_categorical('use_batch_norm', [True, False])
        weight_decay = 0.0 if use_batch_norm else trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)
        
        # Create model
        model = MLPClassifier(
            input_size=self.config['model']['input_size'],
            hidden_layers=hidden_layers,
            num_classes=self.config['model']['num_classes'],
            dropout_rate=dropout_rate,
            use_batch_norm=use_batch_norm
        )
        
        # Create optimizer
        optimizer = getattr(torch.optim, self.config['training']['optimizer_choice'])(
            model.parameters(),
            lr=lr,
            weight_decay=weight_decay
        )
        
        trial_params = {
            'n_layers': n_layers,
            'hidden_layers': hidden_layers,
            'lr': lr,
            'dropout_rate': dropout_rate,
            'use_batch_norm': use_batch_norm,
            'weight_decay': weight_decay
        }
        
        return model, optimizer, trial_params
    
    def objective(self, trial, train_loader, val_loader):
        model, optimizer, trial_params = self.create_model_and_optimizer(trial)
        criterion = getattr(nn, self.config['training']['loss_function'])()
        
        trainer = PyTorchTrainer(
            model, criterion, optimizer,
            device=self.config['training']['device']
        )
        
        patience = self.config['optimization']['early_stopping']['patience']
        min_delta = self.config['optimization']['early_stopping']['min_delta']
        best_metric = float('-inf')
        patience_counter = 0
        last_metric = float('-inf')
        
        # Add warm-up period
        warm_up_epochs = 3
        running_metrics = []
        
        for epoch in range(self.config['training']['epochs']):
            trainer.train_epoch(train_loader)
            _, accuracy, f1 = trainer.evaluate(val_loader)
            
            metric = f1 if self.config['training']['optimization_metric'] == 'f1' else accuracy
            trial.report(metric, epoch)
            
            running_metrics.append(metric)
            if len(running_metrics) > 3:
                running_metrics.pop(0)
            
            # Early stopping logic
            if metric > best_metric + min_delta:
                best_metric = metric
                patience_counter = 0
                
                if metric > self.best_trial_value:
                    self.best_trial_value = metric
                    self.save_best_model(model, optimizer, metric, trial_params)
            else:
                patience_counter += 1
            
            # Modified pruning logic with warm-up and relative threshold
            if epoch >= warm_up_epochs:
                avg_metric = sum(running_metrics) / len(running_metrics)
                relative_deterioration = (best_metric - avg_metric) / (best_metric + 1e-8)
                
                if relative_deterioration > 0.3:  # 30% deterioration threshold
                    raise optuna.TrialPruned("Trial pruned due to significant metric deterioration")
            
            last_metric = metric
            
            if patience_counter >= patience:
                break
        
        return best_metric
    
    def tune(self, train_loader, val_loader):
        study = optuna.create_study(
            direction="maximize",
            pruner=optuna.pruners.MedianPruner()
        )
        
        study.optimize(
            lambda trial: self.objective(trial, train_loader, val_loader),
            n_trials=self.config['optimization']['n_trials']
        )
        
        return study.best_trial, study.best_params

def restore_best_model(config):
    """Utility function to restore the best model and its optimizer."""
    checkpoint = torch.load(config['model']['save_path'], weights_only=True)
    
    # Create model with saved hyperparameters
    model = MLPClassifier(
        input_size=config['model']['input_size'],
        hidden_layers=checkpoint['hyperparameters']['hidden_layers'],
        num_classes=config['model']['num_classes'],
        dropout_rate=checkpoint['hyperparameters']['dropout_rate'],
        use_batch_norm=checkpoint['hyperparameters']['use_batch_norm']
    )
    
    # Create optimizer
    optimizer = getattr(torch.optim, checkpoint['optimizer_name'])(
        model.parameters(),
        lr=checkpoint['hyperparameters']['lr'],
        weight_decay=checkpoint['hyperparameters'].get('weight_decay', 0.0)
    )
    
    # Load states
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    return {
        'model': model,
        'optimizer': optimizer,
        'metric_name': checkpoint['metric_name'],
        'metric_value': checkpoint['metric_value'],
        'hyperparameters': checkpoint['hyperparameters']
    }

def save_best_params_to_config(config_path, best_trial, best_params):
    """Save best parameters to config file."""
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    
    # Create best_model section if it doesn't exist
    if 'best_model' not in config:
        config['best_model'] = {}
    
    # Format parameters for config
    hidden_layers = [best_params[f'hidden_layer_{i}'] for i in range(best_params['n_layers'])]
    
    config['best_model'].update({
        'hidden_layers': hidden_layers,
        'dropout_rate': best_params['dropout_rate'],
        'learning_rate': best_params['lr'],
        'use_batch_norm': best_params['use_batch_norm'],
        'weight_decay': best_params.get('weight_decay', 0.0),
        'best_metric_name': config['training']['optimization_metric'],
        'best_metric_value': best_trial.value
    })
    
    with open(config_path, 'w') as f:
        yaml.dump(config, f, default_flow_style=False)

def train_final_model(config, train_loader, val_loader):
    """Train model using parameters from config."""
    best_model_config = config['best_model']
    
    final_model = MLPClassifier(
        input_size=config['model']['input_size'],
        hidden_layers=best_model_config['hidden_layers'],
        num_classes=config['model']['num_classes'],
        dropout_rate=best_model_config['dropout_rate'],
        use_batch_norm=best_model_config['use_batch_norm']
    )
    
    criterion = getattr(nn, config['training']['loss_function'])()
    optimizer = getattr(torch.optim, config['training']['optimizer_choice'])(
        final_model.parameters(),
        lr=best_model_config['learning_rate'],
        weight_decay=best_model_config['weight_decay']
    )
    
    final_trainer = PyTorchTrainer(
        final_model, criterion, optimizer,
        device=config['training']['device'],
        verbose=True
    )
    
    return final_trainer.train(
        train_loader, 
        val_loader, 
        config['training']['epochs'],
        metric=config['training']['optimization_metric']
    )

def set_seed(seed):
    """Set seed for reproducibility."""
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

class CPUOptimizer:
    """Handles CPU-specific optimizations for PyTorch training."""
    
    def __init__(self, config):
        self.config = config
        self.logger = logging.getLogger('CPUOptimizer')
        self.cpu_info = cpuinfo.get_cpu_info()
        
    def detect_cpu_features(self):
        """Detect CPU features and capabilities."""
        features = {
            'processor': self.cpu_info.get('brand_raw', 'Unknown'),
            'architecture': self.cpu_info.get('arch', 'Unknown'),
            'cores': psutil.cpu_count(logical=False),
            'threads': psutil.cpu_count(logical=True),
            'avx512': 'avx512' in self.cpu_info.get('flags', []),
            'avx2': 'avx2' in self.cpu_info.get('flags', []),
            'mkl': hasattr(torch, 'backends') and hasattr(torch.backends, 'mkl') and torch.backends.mkl.is_available(),
            'ipex': hasattr(torch, 'xpu') or hasattr(torch, 'ipex')
        }
        return features
        
    def configure_optimizations(self):
        """Configure CPU-specific optimizations based on detected features."""
        features = self.detect_cpu_features()
        optimizations = {}
        
        # Configure number of threads
        if self.config['training']['cpu_optimization']['num_threads'] == 'auto':
            optimizations['num_threads'] = features['threads']
        else:
            optimizations['num_threads'] = self.config['training']['cpu_optimization']['num_threads']
        
        # Configure MKL-DNN
        optimizations['enable_mkldnn'] = (
            features['avx512'] or features['avx2']
        ) and self.config['training']['cpu_optimization']['enable_mkldnn']
        
        # Configure data types
        optimizations['use_bfloat16'] = (
            features['avx512'] and 
            self.config['training']['cpu_optimization']['use_bfloat16']
        )
        
        # Set thread configurations
        torch.set_num_threads(optimizations['num_threads'])
        if hasattr(torch, 'set_num_interop_threads'):
            torch.set_num_interop_threads(min(4, optimizations['num_threads']))
        
        # Enable MKL-DNN if available
        if optimizations['enable_mkldnn']:
            torch.backends.mkldnn.enabled = True
        
        self.log_optimization_config(features, optimizations)
        return optimizations
        
    def log_optimization_config(self, features, optimizations):
        """Log CPU features and applied optimizations."""
        self.logger.info("CPU Configuration:")
        self.logger.info(f"Processor: {features['processor']}")
        self.logger.info(f"Architecture: {features['architecture']}")
        self.logger.info(f"Physical cores: {features['cores']}")
        self.logger.info(f"Logical threads: {features['threads']}")
        self.logger.info("\nCPU Features:")
        self.logger.info(f"AVX-512 support: {features['avx512']}")
        self.logger.info(f"AVX2 support: {features['avx2']}")
        self.logger.info(f"MKL support: {features['mkl']}")
        self.logger.info(f"IPEX support: {features['ipex']}")
        self.logger.info("\nApplied Optimizations:")
        self.logger.info(f"Number of threads: {optimizations['num_threads']}")
        self.logger.info(f"MKL-DNN enabled: {optimizations['enable_mkldnn']}")
        self.logger.info(f"BFloat16 enabled: {optimizations['use_bfloat16']}")

def main():
    config_path = 'config.yaml'
    config = load_config(config_path)
    
    # Set up logging
    setup_logger()
    
    # Initialize CPU optimization
    cpu_optimizer = CPUOptimizer(config)
    optimizations = cpu_optimizer.configure_optimizations()
    
    # Set seed for reproducibility
    set_seed(config['training']['seed'])
    
    # Create datasets and dataloaders
    train_df = pd.read_csv(config['data']['train_path'])
    val_df = pd.read_csv(config['data']['val_path'])
    train_dataset = CustomDataset(train_df, config['data']['target_column'])
    val_dataset = CustomDataset(val_df, config['data']['target_column'])
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=config['training']['batch_size'],
        shuffle=True
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=config['training']['batch_size'],
        shuffle=False
    )
    
    # If best parameters don't exist in config, run hyperparameter tuning
    if 'best_model' not in config:
        tuner = HyperparameterTuner(config)
        best_trial, best_params = tuner.tune(train_loader, val_loader)
        save_best_params_to_config(config_path, best_trial, best_params)
        # Reload config with saved parameters
        config = load_config(config_path)
    
    print("\nBest model parameters from config:")
    for key, value in config['best_model'].items():
        print(f"    {key}: {value}")
    
    # Restore best model from checkpoint
    print("\nRestoring best model from checkpoint...")
    restored = restore_best_model(config)
    model = restored['model']
    optimizer = restored['optimizer']
    
    # Create criterion for evaluation
    criterion = getattr(nn, config['training']['loss_function'])()
    
    # Create trainer for evaluation
    trainer = PyTorchTrainer(
        model, criterion, optimizer,
        device=config['training']['device'],
        verbose=True
    )
    
    # Evaluate restored model
    print("\nEvaluating restored model on validation set...")
    val_loss, val_accuracy, val_f1 = trainer.evaluate(val_loader)
    
    metric_name = config['training']['optimization_metric']
    metric_value = val_f1 if metric_name == 'f1' else val_accuracy
    
    print(f"\nRestored model performance:")
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.2f}%")
    print(f"Validation F1-Score: {val_f1:.4f}")
    print(f"\nBest {metric_name.upper()} from tuning: {restored['metric_value']:.4f}")
    print(f"Current {metric_name.upper()}: {metric_value:.4f}")

if __name__ == "__main__":
    main()

[I 2025-01-06 04:52:02,437] A new study created in memory with name: no-name-827af9f8-5f2b-48c7-826c-a8cd0d656d7e
[I 2025-01-06 04:52:03,568] Trial 0 finished with value: 32.5 and parameters: {'n_layers': 4, 'hidden_layer_0': 349, 'hidden_layer_1': 217, 'hidden_layer_2': 111, 'hidden_layer_3': 253, 'lr': 0.0001528846969423591, 'dropout_rate': 0.25632740921438174, 'use_batch_norm': False, 'weight_decay': 0.00023685769920732717}. Best is trial 0 with value: 32.5.
[I 2025-01-06 04:52:04,436] Trial 1 finished with value: 33.0 and parameters: {'n_layers': 4, 'hidden_layer_0': 262, 'hidden_layer_1': 492, 'hidden_layer_2': 156, 'hidden_layer_3': 104, 'lr': 0.07340111719517645, 'dropout_rate': 0.29457323840596084, 'use_batch_norm': False, 'weight_decay': 0.0018455239279866163}. Best is trial 1 with value: 33.0.
[I 2025-01-06 04:52:04,843] Trial 2 pruned. Trial pruned due to significant metric deterioration
[I 2025-01-06 04:52:06,080] Trial 3 finished with value: 29.5 and parameters: {'n_layers


Best model parameters from config:
    best_metric_name: f1_score
    best_metric_value: 36.5
    dropout_rate: 0.20115280210708425
    hidden_layers: [177, 487, 217, 32]
    learning_rate: 1.0397486776002072e-05
    use_batch_norm: False
    weight_decay: 0.00027323564506194205

Restoring best model from checkpoint...

Evaluating restored model on validation set...

Restored model performance:
Validation Loss: 1.3803
Validation Accuracy: 36.50%
Validation F1-Score: 0.2620

Best F1_SCORE from tuning: 36.5000
Current F1_SCORE: 36.5000
